In [1]:
import asyncio
import os
import re
import csv
import ast
from tqdm import tqdm
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

## Creating keywords from summaries

In [2]:
prompt_template = """
You are a highly-efficient expert of knowledge databases.

Your task is to assign high-level keyword categories to summaries of podcast segments you receive.
These keywrods will be used by users to search through podcast material. 
That means, the keyword categories you assign should be easy to search through and accurate.

As an input, you will receive a text with summary.

As an output, I want Python list of three to five elements with keyword categories (each category should consists of 1-2 words).
Output only the list, no other text nor description.


SUMMARY:
```
{text}
```


ASSIGNED KEYWORDS (Python list):"""

In [3]:
async def async_keyword_assign(chain, full_path, retry_number=5):
    tries_number = 0

    with open(full_path) as f:
        text = f.read()
    
    while tries_number <= retry_number:
        try:
            keywords = await chain.arun(text)
            keyword_list = ast.literal_eval(keywords)
            return keyword_list
        except:
            tries_number+=1
            print(f"Extraction failed, trying again. Tries: {tries_number}")
    return []
        


In [4]:
def normalize_string(s):
    return re.sub(r'[^a-zA-Z0-9]', '_', s).lower()

async def process_batch(chain, batch, input_directory, output_directory, segment_urls, pattern):
    tasks = []
    for filename in batch:
        full_path = os.path.join(input_directory, filename)
        save_path = os.path.join(output_directory, filename)
        task = process_file(chain, filename, full_path, save_path, segment_urls, pattern)
        tasks.append(task)
    await asyncio.gather(*tasks)


async def process_file(chain, filename, input_path, output_path, segment_urls, pattern):
    match = re.search(pattern, filename)
    if match:
        # Extracting meta info
        guest_name = match.group(1).replace(' ', '_')
        episode_name = match.group(2)
        episode_num = match.group(3)
        segment_name = match.group(4).strip()

        # Extracting video info
        video_name = f"{guest_name}_ {episode_name} _ Lex Fridman Podcast #{episode_num}"
        segment_url = segment_urls.get((normalize_string(video_name), normalize_string(segment_name)), '')

        # Creating the embed iframe
        if segment_url:
            video_id = segment_url.split('=')[1].split('&')[0]
            start_time = segment_url.split('&t=')[1].rstrip('s')
            hashtags = f"#{guest_name} #episode_{episode_num} "
            segment_embed = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}?start={start_time}" frameborder="0" allowfullscreen></iframe>\n\n'
        else:
            hashtags = f"#{guest_name} #episode_{episode_num} "
            segment_embed = ''

        # Extract keywords
        keywords = await async_keyword_assign(chain, input_path)
        keyword_hashtags = ' '.join([f'#{keyword.replace(" ", "_")}' for keyword in keywords])
        
        # Read the content and write the modified content
        with open(input_path, 'r') as file:
            content = file.read()
        with open(output_path, 'w') as file:
            file.write(hashtags + keyword_hashtags + '\n\n' + segment_embed + content)


async def modify_summaries(input_directory='summaries', output_directory='hashtaged_keyword_summaries', metadata_file='segments_metadata.csv', prompt_template='', model_name="gpt-3.5-turbo", batch_size=50):
    os.makedirs(output_directory, exist_ok=True)

    # Initialize the chain for keyword extraction
    llm = ChatOpenAI(model_name=model_name)
    KEYWORD_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    chain = LLMChain(llm=llm, prompt=KEYWORD_PROMPT)
    
    segment_urls = {}
    with open(metadata_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            key = (normalize_string(row['video_name']), normalize_string(row['segment_name']))
            segment_urls[key] = row['segment_url']

    pattern = r"\(Summary\) Episode - ([^_]+)_ ([^_]+) _ Lex Fridman Podcast #(\d+) Segment - ([^\(]+)"

    filenames = [filename for filename in os.listdir(input_directory) if not filename.startswith('.') and not os.path.exists(os.path.join(output_directory, filename))]
    for i in tqdm(range(0, len(filenames), batch_size)):
        batch = filenames[i:i+batch_size]
        await process_batch(chain, batch, input_directory, output_directory, segment_urls, pattern)
        await asyncio.sleep(25)  # Waiting 30 seconds after processing each

In [5]:
await modify_summaries(prompt_template=prompt_template, batch_size=70)

  0%|          | 0/22 [00:00<?, ?it/s]

Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2


  5%|▍         | 1/22 [00:28<09:57, 28.48s/it]

Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. T

  9%|▉         | 2/22 [01:04<10:52, 32.63s/it]

...

 91%|█████████ | 20/22 [30:57<05:31, 165.92s/it]

Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 5
Extraction failed, trying again. T

 95%|█████████▌| 21/22 [31:29<02:05, 125.59s/it]

Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 1
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 2
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 5
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. Tries: 5
Extraction failed, trying again. Tries: 3
Extraction failed, trying again. Tries: 4
Extraction failed, trying again. T

100%|██████████| 22/22 [32:02<00:00, 87.37s/it] 
